In [22]:
#このファイルはvoicevox apiとopenai apiを使用することを想定しています。

In [44]:
import os
import openai
import re
import time
import requests
import json
import asyncio
import winsound
import shutil
import pyaudio
import wave

openai.api_key="キーを入力してください。"
openai.api_base = "https://api.openai.iniad.org/api/v1"

# voiceフォルダが存在するか確認
if os.path.exists("voice"):
    # フォルダを削除
    shutil.rmtree("voice")

# フォルダを作成
os.makedirs("voice")

In [49]:
def make_voice(text,num):
    #hostにvoicevoxサーバーのipを入れて下さい。
    host = '127.0.0.1'
    port = '50021'
    params = (
            ('text', text),
            ('speaker', 14),
        )
    
    # 音声合成用のクエリ作成
    query = requests.post(
        f'http://{host}:{port}/audio_query',
        params=params
    )
    
    # 音声合成を実施
    synthesis = requests.post(
        f'http://{host}:{port}/synthesis',
        headers = {"Content-Type": "application/json"},
        params = params,
        data = json.dumps(query.json())
    )
    
    #voiceフォルダに生成した音声ファイルを格納します。
    with open(f"voice/{num}.wav",mode="wb") as f:
        f.write(synthesis.content)

In [46]:
#pyaudioでの音声再生用関数です。windows環境でない方はこちらをお使いください。
def play_pyaudio(path, chunk=2048):
    audio = None
    stream = None

    try:
        wf = wave.open(path, 'rb')
    except IOError as e:
        print(f"Error: {e}")
        return

    try:
        audio = pyaudio.PyAudio()
        stream = audio.open(
            format=audio.get_format_from_width(wf.getsampwidth()),
            channels=wf.getnchannels(),
            rate=wf.getframerate(),
            output=True,
        )

        data = wf.readframes(chunk)
        while len(data) > 0:
            stream.write(data)
            data = wf.readframes(chunk)

    except Exception as e:
        print(f"Error: {e}")

    finally:
        if stream is not None:
            stream.stop_stream()
            stream.close()

        if audio is not None:
            audio.terminate()

In [47]:
def playwav(i):
    #max_attemptsは音声ファイルの生成時間によって変動します。調整してください。
    max_attempts = 6
    cnt = 0
    if i == 0:
        while True:
            path = os.path.abspath(f'voice/{i}.wav')
            if os.path.exists(path):
                with open(path, 'rb') as f:
                    data = f.read()
                    winsound.PlaySound(data, winsound.SND_MEMORY)
                    
                #windows以外の動作環境ならばこちらを使ってください。
                #play_pyaudio(path)
                i+=1
                cnt = 0
                break
            else:
                time.sleep((cnt/10)**2)
                cnt += 1
            
    while True:
        path = os.path.abspath(f'voice/{i}.wav')
        if os.path.exists(path):
            with open(path, 'rb') as f:
                data = f.read()
                winsound.PlaySound(data, winsound.SND_MEMORY)
            
            #windows以外の動作環境ならばこちらを使ってください。
            #play_pyaudio(path) 
            i+=1
            cnt = 0
            
        else:
            if cnt >= max_attempts:
                for filename in os.listdir("voice"):
                    os.remove(f'voice/{filename}')
                if not os.path.isdir("voice"):
                    os.mkdir("voice")
                print("End")
                break
                
            else:
                time.sleep((cnt/10)**2)
                print(f"File:{i} Attempt number:{cnt}")
                cnt += 1

In [48]:
#ChatGPTでの文章生成
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {'role': 'user', 'content': "こんにちは"}
    ],
    stream=True
)

collected_chunks = []
collected_messages = []
i = 0

#オーディオ再生関数
asyncio.new_event_loop().run_in_executor(None, playwav, 0)

for chunk in response:
    collected_chunks.append(chunk)
    chunk_message = chunk['choices'][0]['delta'].get('content', '')
    
    #ChatGPTの回答をセンテンスごとに区切ります。
    if re.match(r"[!?！？、。]", chunk_message):
        sentence = ''.join(collected_messages)
        #voicevox api で音声ファイルを生成します。
        asyncio.new_event_loop().run_in_executor(None, make_voice, sentence,i)
        print(sentence)
        collected_messages = []
        i += 1
    else:
        collected_messages.append(chunk_message)

こんにちは
いつもお忙しい中
私に質問や相談があればどうぞ
お手伝いできることがあれば喜んでお答えします
File:4 Attempt number:0
File:4 Attempt number:1
File:4 Attempt number:2
File:4 Attempt number:3
File:4 Attempt number:4
File:4 Attempt number:5
End
